# State Machine for the Kitchen Timer

This is material for the lecture, not part of the mandatory preparation.

In [5]:
from stmpy import Machine, Driver

import ipywidgets as widgets
from IPython.display import display

class KitchenTimer:
    
    def on_button_press(self, b):
        self.stm.send('switch') # <---- here we send a signal
            
    def __init__(self):
        # load images and store them
        self.on_60 = open("images/timer/on_60.jpg", "rb").read()
        self.off_60 = open("images/timer/off_60.jpg", "rb").read()
        self.on_45 = open("images/timer/on_45.jpg", "rb").read()
        self.off_45 = open("images/timer/off_45.jpg", "rb").read()
        self.on_30 = open("images/timer/on_30.jpg", "rb").read()
        self.off_30 = open("images/timer/off_30.jpg", "rb").read()
        self.on_15 = open("images/timer/on_15.jpg", "rb").read()
        self.off_15 = open("images/timer/off_15.jpg", "rb").read()
        self.plug_on = open("images/timer/plug_on.jpg", "rb").read()
        self.plug_off = open("images/timer/plug_off.jpg", "rb").read()
        
        self.led_15 = widgets.Image(value=self.off_15, format='jpg', width=50, height=50)
        self.led_30 = widgets.Image(value=self.off_30, format='jpg', width=50, height=50)
        self.led_45 = widgets.Image(value=self.off_45, format='jpg', width=50, height=50)
        self.led_60 = widgets.Image(value=self.off_60, format='jpg', width=50, height=50)
        
        left_box = widgets.VBox([self.led_60, self.led_45])
        right_box = widgets.VBox([self.led_15, self.led_30])
        box = widgets.HBox([left_box, right_box])
        self.plug = widgets.Image(value=self.plug_off, format='jpg', width=100, height=100)
        
        # display the user interface
        # a button
        self.button = widgets.Button(description="Button")
        self.button.on_click(self.on_button_press)
        
        display(box, self.button, self.plug)

    def set_led(self, led15: bool, led30: bool, led45: bool, led60: bool):
        self.switch_led('15', led15)
        self.switch_led('30', led30)
        self.switch_led('45', led45)
        self.switch_led('60', led60)
        
    
    def switch_led(self, led, on):
        if led == '15' and on: self.led_15.set_trait(name='value', value=self.on_15)
        if led == '15' and not on: self.led_15.set_trait(name='value', value=self.off_15) 
        if led == '30' and on: self.led_30.set_trait(name='value', value=self.on_30)
        if led == '30' and not on: self.led_30.set_trait(name='value', value=self.off_30) 
        if led == '45' and on: self.led_45.set_trait(name='value', value=self.on_45)
        if led == '45' and not on: self.led_45.set_trait(name='value', value=self.off_45) 
        if led == '60' and on: self.led_60.set_trait(name='value', value=self.on_60)
        if led == '60' and not on: self.led_60.set_trait(name='value', value=self.off_60) 
            
    def switch_plug(self, on):
        if on: self.plug.set_trait(name='value', value=self.plug_on)
        else: self.plug.set_trait(name='value', value=self.plug_off) 

In [6]:
timer = KitchenTimer()

transitions = [
	{
		"source": "initial",
		"target": "off",
	},
	{
		"trigger": "switch",
		"source": "off",
		"target": "on_15",
		"effects": "switch_plug(True)"
	},
	{
		"trigger": "switch",
		"source": "on_15",
		"target": "on_30",
	},
	{
		"trigger": "switch",
		"source": "on_30",
		"target": "on_45",
	},
	{
		"trigger": "switch",
		"source": "on_45",
		"target": "on_60",
	},
	{
		"trigger": "switch",
		"source": "on_60",
		"target": "off",
	},
	{
		"trigger": "t",
		"source": "on_15",
		"target": "off",
	},
	{
		"trigger": "t",
		"source": "on_30",
		"target": "on_15",
	},
	{
		"trigger": "t",
		"source": "on_45",
		"target": "on_30",
	},
	{
		"trigger": "t",
		"source": "on_60",
		"target": "on_45",
	},
]

states = [
	{
		"name": "off",
		"entry": "switch_plug(False); set_led(False, False, False, False); stop_timer('t')",
	},
	{
		"name": "on_15",
		"entry": "start_timer('t', 900); set_led(True, False, False, False)"
	},
	{
		"name": "on_30",
		"entry": "start_timer('t', 900); set_led(True, True, False, False)"
	},
	{
		"name": "on_45",
		"entry": "start_timer('t', 900); set_led(True, True, True, False)"
	},
	{
		"name": "on_60",
		"entry": "start_timer('t', 900); set_led(True, True, True, True)",
	},
]

machine = Machine(
	name="kitchen_timer",
	transitions=transitions,
	obj=timer,
	states=states,
)
timer.stm = machine

driver = Driver()
driver.add_machine(machine)
driver.start()

Button(description='Button', style=ButtonStyle())

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\xc8\x00\xc8\x00\x00\xff\xe1\x00\x98Exif\x00\x00…